In [ ]:
%%sh
pip install -q sagemaker --upgrade

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'huggingface/demo-customer-reviews'

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                     role=sagemaker.get_execution_role(),
                                     instance_type='ml.m5.2xlarge',
                                     instance_count=1)

In [ ]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing.py',
    
    outputs=[
        ProcessingOutput(
                output_name='training',
                source='/opt/ml/processing/output/training'),
        ProcessingOutput(
                output_name='validation',
                source='/opt/ml/processing/output/validation')
    ],
    
    arguments=[
               '--threshold', '4',
               '--s3-bucket', bucket,
               '--s3-prefix', prefix
    ]
)

In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])